# Exemple de notebook : Utilisation du package IBIS pour pré-filtrer, faire des jointures et aggréger avant de charger les données avec pandas

Avec la collecte de plusieurs années d’historique des données SISE, les volumes de données disponibles dans les tables edc_communes, edc_prelevements et edc_resultats vont significativement augmenter. Cette situationsituation pose des défis en termes de performances pour le chargement des données.

En particulier, une approche consistant à charger une table entière dans un DataFrame Pandas avec une commande comme :

```python
df = con.table("edc_resultats").df()
```

peut devenir lente et gourmande en mémoire.

Ce notebook a pour objectif de montrer :

- Comment utiliser [Ibis](https://ibis-project.org/tutorials/getting_started) pour interroger et filtrer efficacement les données avant de les charger dans Pandas.
- Des exemples concrets adaptés à vos besoins d'analyse, comme le filtrage par dates, jointures entre tables ou la sélection de paramètres spécifiques.

In [55]:
# Connexion à la base de données via ibis
import ibis
ibis_con = ibis.connect("./../../database/data.duckdb", read_only=True)

Pour commencer, listons les tables disponibles dans la base de données :

In [56]:
ibis_con.list_tables()

['edc_communes', 'edc_prelevements', 'edc_resultats']

L'inspection des colonnes d'une table peut se faire directement avec Ibis.

In [57]:
communes_table = ibis_con.table("edc_communes")
prelevements_table = ibis_con.table("edc_prelevements")
resultats_table = ibis_con.table("edc_resultats")

display(communes_table)
display(prelevements_table)
display(resultats_table)

DatabaseTable: edc_communes
  inseecommune string
  nomcommune   string
  quartier     string
  cdreseau     string
  nomreseau    string
  debutalim    date
  de_partition int32

DatabaseTable: edc_prelevements
  cddept                     string
  cdreseau                   string
  inseecommuneprinc          string
  nomcommuneprinc            string
  cdreseauamont              string
  nomreseauamont             string
  pourcentdebit              string
  referenceprel              string
  dateprel                   date
  heureprel                  string
  conclusionprel             string
  ugelib                     string
  distrlib                   string
  moalib                     string
  plvconformitebacterio      string
  plvconformitechimique      string
  plvconformitereferencebact string
  plvconformitereferencechim string
  de_partition               int32

DatabaseTable: edc_resultats
  cddept                   string
  referenceprel            string
  cdparametresiseeaux      string
  cdparametre              int64
  libmajparametre          string
  libminparametre          string
  libwebparametre          string
  qualitparam              string
  insituana                string
  rqana                    string
  cdunitereferencesiseeaux string
  cdunitereference         string
  limitequal               string
  refqual                  string
  valtraduite              float64
  casparam                 string
  referenceanl             string
  de_partition             int32

## Filtres

1. Filtrer sur les prélévements de 2024

In [58]:
prelevements_2024 = prelevements_table.filter(prelevements_table.dateprel >= "2024-01-01")

# On peut mettre une limite avant d'éxecuter la requête.
df_prelevements_2024 = prelevements_2024.limit(50).execute()
df_prelevements_2024.head(2)

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition
0,001,001000003,01007,AMBRONAY,None,None,None,00100139034,2024-01-23,11h27,Eau d'alimentation conforme aux limites de qua...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,N,C,2024
1,001,001000003,01007,AMBRONAY,None,None,None,00100139393,2024-02-13,12h49,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024


2. Filtrer sur les prélévements non conformes en 2024



In [59]:
non_conforme_query = (
    (prelevements_table.plvconformitebacterio == "N") |
    (prelevements_table.plvconformitechimique == "N") |
    (prelevements_table.plvconformitereferencebact == "N") |
    (prelevements_table.plvconformitereferencechim == "N")
)

non_conformes_2024 = prelevements_2024.filter(
    non_conforme_query
)
f"En 2024, il y a eu {non_conformes_2024.count().execute():,.0f} prélévements non conformes au sens d'au moins une des variables suivantes : plvconformitebacterio, plvconformitechimique, plvconformitereferencebact, plvconformitereferencechim ([voir documentation](https://www.data.gouv.fr/fr/datasets/r/36afc708-42dc-4a89-b039-7fde6bcc83d8))".replace(",", " ") 


"En 2024  il y a eu 89 299 prélévements non conformes au sens d'au moins une des variables suivantes : plvconformitebacterio  plvconformitechimique  plvconformitereferencebact  plvconformitereferencechim ([voir documentation](https://www.data.gouv.fr/fr/datasets/r/36afc708-42dc-4a89-b039-7fde6bcc83d8))"

In [60]:
non_conformes_paris = non_conformes_2024.filter(
  (prelevements_table.nomcommuneprinc == "PARIS")
)
f"{non_conformes_paris.count().execute():,.0f} sur la ville de PARIS"

'32 sur la ville de PARIS'

## Selectionner des colonnes avant d'exécuter la requête

In [61]:
selected_columns = non_conformes_2024[["referenceprel", "dateprel", "nomcommuneprinc", "plvconformitebacterio"]]
selected_columns.execute()

,referenceprel,dateprel,nomcommuneprinc,plvconformitebacterio
0,00100142530,2024-10-18,AMBRONAY,N
1,00100141265,2024-07-10,MURS-ET-GELIGNIEUX,N
2,00100142451,2024-10-17,AMBUTRIX,N
3,00100143357,2024-11-22,ANGLEFORT,N
4,00100141194,2024-07-10,ARBOYS EN BUGEY,N
...,...,...,...,...
89294,97600032043,2024-08-12,BOUENI,C
89295,97600032044,2024-08-12,BOUENI,C
89296,97600031309,2024-01-11,PAMANDZI,C
89297,97600031337,2024-01-15,PAMANDZI,C


## Jointure
Joindre edc_prelevements et edc_resultats sur referenceprel pour obtenir les résultats associés à chaque prélèvement :

In [62]:

joined_data = non_conformes_2024.join(
    resultats_table,
    non_conformes_2024.referenceprel == resultats_table.referenceprel
)[["referenceprel", "dateprel", "nomcommuneprinc", "libmajparametre","insituana", "rqana", "cdunitereferencesiseeaux"]]

joined_data.execute()

,referenceprel,dateprel,nomcommuneprinc,libmajparametre,insituana,rqana,cdunitereferencesiseeaux
0,01100168135,2024-09-17,COUSTOUGE,"DICHLOROÉTHANE-1,2",L,"<0,20",µg/L
1,01100168135,2024-09-17,COUSTOUGE,"2,4,5-T",L,"<0,020",µg/L
2,01100168135,2024-09-17,COUSTOUGE,"2,4-D",L,"<0,020",µg/L
3,01100168135,2024-09-17,COUSTOUGE,"2,6 DICHLOROBENZAMIDE",L,"<0,005",µg/L
4,01100168135,2024-09-17,COUSTOUGE,HYDROXYCARBOFURAN-3,L,"<0,005",µg/L
...,...,...,...,...,...,...,...
6024762,97200106851,2024-01-15,GROS-MORNE,PH,L,"6,8",unité pH
6024763,97200106851,2024-01-15,GROS-MORNE,PH,T,"6,9",unité pH
6024764,97200106851,2024-01-15,GROS-MORNE,SAVEUR (QUALITATIF),T,Aucun changement anormal,SANS OBJET
6024765,97200106851,2024-01-15,GROS-MORNE,TEMPÉRATURE DE L'EAU (DOM),T,"24,3",°C


## Groupby et aggregats
Nombre total de prélèvements non conforme par commune en 2024

In [63]:
agg_data = non_conformes_2024.group_by("nomcommuneprinc").aggregate(
    nb_prelevements_non_conformes=non_conformes_2024.referenceprel.count()
)
agg_data.execute().sort_values("nb_prelevements_non_conformes", ascending=False)

,nomcommuneprinc,nb_prelevements_non_conformes
2121,MARSEILLE,2016
5300,CAPESTERRE-BELLE-EAU,592
4145,AJACCIO,374
5310,SAINT-DENIS,344
12808,PREMESQUES,294
...,...,...
13,VOULON,1
11,POIROUX,1
9,NOIRMOUTIER-EN-L'ILE,1
7,CHATEAUNEUF-DU-PAPE,1
